In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import broadcast
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [128]:
df_hpns = spark.read.option("header", "true").csv("C:/Users/Conaf/Documents/BootCamp/BigData-Processing/big-data-processing/Proyecto-Final/Datasets/world-happiness-report.csv")
df_21 = spark.read.option("header", "true").csv("C:/Users/Conaf/Documents/BootCamp/BigData-Processing/big-data-processing/Proyecto-Final/Datasets/world-happiness-report-2021.csv")

df_hpns.printSchema()
df_21.printSchema()

df_hpns = df_hpns.withColumn("year", col("year").cast("int"))
df_hpns = df_hpns.withColumn("Life Ladder", col("Life Ladder").cast("float"))
df_hpns = df_hpns.withColumn("Log GDP per capita", col("Log GDP per capita").cast("float"))
df_hpns = df_hpns.withColumn("Social support", col("Social support").cast("float"))
df_hpns = df_hpns.withColumn("Healthy life expectancy at birth", col("Healthy life expectancy at birth").cast("float"))
df_hpns = df_hpns.withColumn("Freedom to make life choices", col("Freedom to make life choices").cast("float"))
df_hpns = df_hpns.withColumn("Generosity", col("Generosity").cast("float"))
df_hpns = df_hpns.withColumn("Perceptions of corruption", col("Perceptions of corruption").cast("float"))
df_hpns = df_hpns.withColumn("Positive affect", col("Positive affect").cast("float"))
df_hpns = df_hpns.withColumn("Negative affect", col("Negative affect").cast("float"))

df_21 = df_21.withColumn("Ladder score", col("Ladder score").cast("float"))
df_21 = df_21.withColumn("Standard error of ladder score", col("Standard error of ladder score").cast("float"))
df_21 = df_21.withColumn("upperwhisker", col("upperwhisker").cast("float"))
df_21 = df_21.withColumn("lowerwhisker", col("lowerwhisker").cast("float"))
df_21 = df_21.withColumn("Logged GDP per capita", col("Logged GDP per capita").cast("float"))
df_21 = df_21.withColumn("Social support", col("Social support").cast("float"))
df_21 = df_21.withColumn("Healthy life expectancy", col("Healthy life expectancy").cast("float"))
df_21 = df_21.withColumn("Freedom to make life choices", col("Freedom to make life choices").cast("float"))
df_21 = df_21.withColumn("Generosity", col("Generosity").cast("float"))
df_21 = df_21.withColumn("Perceptions of corruption", col("Perceptions of corruption").cast("float"))
df_21 = df_21.withColumn("Ladder score in Dystopia", col("Ladder score in Dystopia").cast("float"))
df_21 = df_21.withColumn("Explained by: Log GDP per capita", col("Explained by: Log GDP per capita").cast("float"))
df_21 = df_21.withColumn("Explained by: Social support", col("Explained by: Social support").cast("float"))
df_21 = df_21.withColumn("Explained by: Healthy life expectancy", col("Explained by: Healthy life expectancy").cast("float"))
df_21 = df_21.withColumn("Explained by: Freedom to make life choices", col("Explained by: Freedom to make life choices").cast("float"))
df_21 = df_21.withColumn("Explained by: Generosity", col("Explained by: Generosity").cast("float"))
df_21 = df_21.withColumn("Explained by: Perceptions of corruption", col("Explained by: Perceptions of corruption").cast("float"))
df_21 = df_21.withColumn("Dystopia + residual", col("Dystopia + residual").cast("float"))

df_hpns.show()
df_hpns.printSchema()
df_21.show()
df_21.printSchema()



+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
| Afghanistan|2008|      3.724|              7.37|         0.451|                            50.8|                       0.718|     0.168|                    0.882|          0.518|          0.258|
| Afghanistan|2009|      4.402|              7.54|         0.552|                            51.2|                       0.679|      0.19|                     0.85|          0.584|          0.237|
| Afghanistan|2

In [129]:
# #1 País más feliz del 2021

row_maxFeliz = df_21.filter(df_21["Ladder score"] == df_21.agg({"Ladder score":"max"}).collect()[0][0])
countryFeliz = row_maxFeliz.select("Country name")

countryFeliz.show()

+------------+
|Country name|
+------------+
|     Finland|
+------------+



In [130]:
# #2 País más feliz por continente

max_Region = df_21.groupBy("Regional indicator").agg(max("Ladder score")).alias("maxHappy")
union = max_Region.join(df_21, max_Region["max(Ladder score)"] == df_21["Ladder score"])
union_reduc = union.select("Country name", "Ladder score")

final = max_Region.join(union_reduc, max_Region["max(Ladder score)"] == union_reduc["Ladder score"])

final.select("Country name", "Regional indicator", "Ladder score").show()


+--------------------+--------------------+------------+
|        Country name|  Regional indicator|Ladder score|
+--------------------+--------------------+------------+
|             Finland|      Western Europe|       7.842|
|         New Zealand|North America and...|       7.277|
|              Israel|Middle East and N...|       7.157|
|          Costa Rica|Latin America and...|       7.069|
|      Czech Republic|Central and Easte...|       6.965|
|Taiwan Province o...|           East Asia|       6.584|
|           Singapore|      Southeast Asia|       6.377|
|          Uzbekistan|Commonwealth of I...|       6.179|
|           Mauritius|  Sub-Saharan Africa|       6.049|
|               Nepal|          South Asia|       5.269|
+--------------------+--------------------+------------+



In [194]:
# #3 País que más veces ocupó el primer lugar todos los años                     

max_anio = df_hpns.groupBy("year").agg(max("Life Ladder").alias("max_ladder"))

max_ladder_pais = df_hpns.alias('df').join(max_anio.alias('ma'), (col("df.year") == col("ma.year")) & (col('df.Life Ladder') == col('ma.max_ladder')))

max_ladder_count = max_ladder_pais.groupBy("Country name").count()

winner = max_ladder_count.orderBy(col("count").desc()).first()[0]

print("El país que más veces ha ocupado el primer lugar es: " + winner)



El país que más veces ha ocupado el primer lugar es: Denmark


In [140]:
# #4 Puesto de felicidad del país con mayor GDP del 2020

# Filtramos el año 2020

df_2020 = df_hpns.filter(df_hpns["year"] == "2020")
df_2020_ord = df_2020.orderBy(desc("Life Ladder"))

df_2020_ord = df_2020_ord.withColumn("Ranking Life Ladder", monotonically_increasing_id() + 1)
max_value = df_2020_ord.agg(max(col("Log GDP per capita")))

df_2020_ord.select("Country name", "Ranking Life Ladder", "year", "Log GDP per capita").filter(col("Log GDP per capita") == max_value.collect()[0][0]).show()


+------------+-------------------+----+------------------+
|Country name|Ranking Life Ladder|year|Log GDP per capita|
+------------+-------------------+----+------------------+
|     Ireland|                 13|2020|            11.323|
+------------+-------------------+----+------------------+



In [149]:
# #5 En qué porcentaje ha variado a nivel mundial el GDP promedio del 2020 al 2021. ¿Aumentó o disminuyó?

df_2020 = df_hpns.filter(df_hpns["year"] == "2020")

prom_2020 = df_2020.select(mean("Log GDP per capita")).collect()[0][0]

prom_2021 = df_21.select(mean("Logged GDP per capita")).collect()[0][0]

perc_var = ((prom_2020 - prom_2021)/prom_2021)*100

print("En 2020 el GDP promedio era: " + str(prom_2020) + ", mientras que en 2021 fue: " + str(prom_2021) 
      + ". Por tanto, hubo una disminución porcentual en el GDP del " + str(perc_var) + "%")

En 2020 el GDP promedio era: 9.751329579136588, mientras que en 2021 fue: 9.432208032416018. Por tanto, hubo una disminución porcentual en el GDP del 3.383317518271791%


In [169]:
# #6 País con máyor expectativa de vida actual y su valor en 2019

max_value = df_21.agg(max(col("Healthy life expectancy")))
pais_max_vida = df_21.select("Country name").filter(col("Healthy life expectancy") == max_value.collect()[0][0])

exp_19 = df_hpns.select("Healthy life expectancy at birth").filter((col("Country name") == pais_max_vida.collect()[0][0]) & (col("year") == "2019"))

print("El país con mayor esperanza de vida actualmente es " + str(pais_max_vida.collect()[0][0]) 
      + " y en el 2012 su esperanza de vida era de " + str(exp_19.collect()[0][0]) + " años")

El país con mayor esperanza de vida actualmente es Singapore y en el 2012 su esperanza de vida era de 77.0999984741211 años
